In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [ ]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt" #url with formatting to fill in the number of the particular week
    dfs = []
    for week_num in week_nums: #for each week
        file_url = url.format(week_num) #format the url so it grabs that week
        dfs.append(pd.read_csv(file_url)) #append to our empty list
    return pd.concat(dfs) #combine files for each week


start_date = '12/29/2018'
num_per = 24
data_collection_period = pd.date_range(start = start_date, periods = num_per, freq = 'W-SAT')
week_nums = []
for item in data_collection_period:
    week_nums.append(int((str(item).rstrip('0:').lstrip('20').replace('-', '')).strip()))

turnstiles_df = get_data(week_nums) #use our function to get a df with all of our data

In [ ]:
csv_filename = f'mta_data_{week_nums[0]}_to_{week_nums[-1]}.csv'
turnstiles_df.to_csv(csv_filename, header = False, index = False)

In [3]:
engine = create_engine("sqlite:///mta.db")

In [ ]:
all_data = pd.read_sql('SELECT * FROM mta_data;', engine)

In [6]:
turnstile_count_by_station = pd.read_sql('SELECT COUNT(DISTINCT SCP), STATION, LINENAME \
                                        FROM mta_data \
                                        GROUP BY STATION, LINENAME \
                                        ORDER BY COUNT(DISTINCT SCP) DESC;', engine)
turnstile_count_by_station.head()

,COUNT(DISTINCT SCP),STATION,LINENAME
0,60,FULTON ST,2345ACJZ
1,50,34 ST-PENN STA,ACE
2,47,PATH NEW WTC,1
3,41,34 ST-HERALD SQ,BDFMNQRW
4,39,CORTLANDT ST,RNW


In [4]:
station_contains_brooklyn = pd.read_sql("SELECT STATION, LINENAME \
                                        FROM mta_data \
                                        WHERE STATION LIKE '%BROOKLYN%';", engine)
station_contains_brooklyn.head()

,STATION,LINENAME
0,BROOKLYN BRIDGE,456JZ
1,BROOKLYN BRIDGE,456JZ
2,BROOKLYN BRIDGE,456JZ
3,BROOKLYN BRIDGE,456JZ
4,BROOKLYN BRIDGE,456JZ
